In [1]:
# para recarregar a biblioteca automaticamente, e não ter que restartar o kernel a cada mudança
%load_ext autoreload
%autoreload 2

In [2]:
#from analises import bibl_pncp

In [3]:
from analises import bibl_mapa

In [4]:
import pandas as pd
import numpy as np
import math

In [5]:
#bibliotecas para visualização de dados (gráficos)
import matplotlib.pyplot as plt
#para conseguir visualizar os gráficos no notebook
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

In [6]:
df_contrato = pd.read_csv('dados/dadosPNCP_completo_tratado.csv', low_memory=False) #low_memory = False: para corrigir o erro de dtype em algumas colunas

#excluindo colunas não utilizadas
df_contrato.drop(columns=['codigoPaisFornecedor', 'numeroRetificacao', 'niFornecedorSubContratado', 'nomeFornecedorSubContratado', 'tipoPessoaSubContratada', 'unidadeSubRogada', 'orgaoSubRogado', 
                        'identificadorCipi', 'urlCipi'], inplace=True)

In [7]:
df_ceis = pd.read_csv('dados/20231023_CEIS.csv', sep=";", encoding='latin1')

#convertendo cpf/cnpj para string pois estava int
df_ceis['CPF OU CNPJ DO SANCIONADO'] = df_ceis['CPF OU CNPJ DO SANCIONADO'].astype(str)
#convertendo a coluna "DATA INÍCIO SANÇÃO" para o formato "YYYY-MM-DD" (para poder comparar com outras datas - ie: data assinatura contrato)
df_ceis['DATA INÍCIO SANÇÃO'] = pd.to_datetime(df_ceis['DATA INÍCIO SANÇÃO'], format='%d/%m/%Y')

#excluindo colunas não utilizadas
df_ceis.drop(columns=['CADASTRO', 'NOME DO SANCIONADO', 'NOME INFORMADO PELO ÓRGÃO SANCIONADOR', 'NOME FANTASIA - CADASTRO RECEITA', 'NÚMERO DO PROCESSO',
                     'DATA PUBLICAÇÃO', 'PUBLICAÇÃO', 'DETALHAMENTO', 'DATA DO TRÂNSITO EM JULGADO'], inplace=True)

In [8]:
df_cnep = pd.read_csv('dados/20231023_CNEP.csv', sep=";", encoding='latin1')

In [9]:
df_cepim = pd.read_csv('dados/20231023_CEPIM.csv', sep=";", encoding='latin1')

In [10]:
#df_contrato = df_contrato.sort_values(by='dataPublicacaoPncp', ascending=False)

In [11]:
#df_contrato.info()
#df_ceis.info()

In [ ]:
#verificando se os campos que usarei tem valores nulos e precisariam de tratamento: niFornecedor, dataVigenciaInicio e dataVigenciaFim
#NÃO TEM NENHUM REGISTRO COM ESSES CAMPOS NULOS!!! :D
df_contratos_nulos = df_contrato[df_contrato['niFornecedor'].isnull() & df_contrato['dataVigenciaInicio'].isnull() & df_contrato['dataVigenciaFim'].isnull() & df_contrato['dataAssinatura'].isnull()]
df_contratos_nulos

In [ ]:
#verificando se tem linhas duplicadas no dataframe de contratos
#tem 291 linhas! excluir no csv origina!!!
tem_duplicatas = df_contrato.duplicated()
num_duplicatas = tem_duplicatas.sum()
num_duplicatas

In [ ]:
#verificando se os campos que usarei tem valores nulos e precisariam de tratamento: CPF OU CNPJ DO SANCIONADO, RAZÃO SOCIAL - CADASTRO RECEITA, CATEGORIA DA SANÇÃO, 
#FUNDAMENTAÇÃO LEGAL, DATA INÍCIO SANÇÃO, DATA FINAL SANÇÃO, ÓRGÃO SANCIONADOR, UF ÓRGÃO SANCIONADOR
#NÃO TEM NENHUM REGISTRO COM ESSES CAMPOS NULOS!!! :D
df_ceis_nulos = df_ceis[df_ceis['CPF OU CNPJ DO SANCIONADO'].isnull() & df_ceis['RAZÃO SOCIAL - CADASTRO RECEITA'].isnull() & df_ceis['CATEGORIA DA SANÇÃO'].isnull() & 
                df_ceis['FUNDAMENTAÇÃO LEGAL'].isnull() & df_ceis['DATA INÍCIO SANÇÃO'].isnull() & df_ceis['DATA FINAL SANÇÃO'].isnull() &
                df_ceis['ÓRGÃO SANCIONADOR'].isnull() & df_ceis['UF ÓRGÃO SANCIONADOR'].isnull()]
df_ceis_nulos 

In [ ]:
#verificando se tem linhas duplicadas no dataframe do CEIS
#NÃO TEM NENHUMA LINHA DUPLICADA!!!! :D
tem_duplicatas = df_ceis.duplicated()
num_duplicatas = tem_duplicatas.sum()
num_duplicatas

In [16]:
#df_filtrado = df_contrato[df_contrato['numeroControlePNCP'] == "18431312000115-2-000161/2023"]
#df_filtrado['nomeRazaoSocialFornecedor']

#df_contrato['niFornecedor'].dtypes
#df_ceis['CPF OU CNPJ DO SANCIONADO'].dtypes

In [17]:
#criando df com os contratos que possuem CNPJ/CPF que constam no csv do CEIS (inner join): - 13293 registros
df_sancoes_cnpj = pd.merge(df_contrato, df_ceis, how='inner', left_on='niFornecedor', right_on='CPF OU CNPJ DO SANCIONADO')

#aplicando os filtros para retornar apenas contratos assiandos em data dentro do perído de vigência da sanção
condicao1 = ( df_sancoes_cnpj['DATA FINAL SANÇÃO'].isnull()  & (df_sancoes_cnpj['DATA INÍCIO SANÇÃO'] <= df_sancoes_cnpj['dataAssinatura']) )  #6 ocorrências
condicao2 = ( df_sancoes_cnpj['DATA FINAL SANÇÃO'].notnull() & ( (df_sancoes_cnpj['dataAssinatura'] >= df_sancoes_cnpj['DATA INÍCIO SANÇÃO']) & #2.838 ocorrências
                                                                (df_sancoes_cnpj['dataAssinatura'] <= df_sancoes_cnpj['DATA FINAL SANÇÃO']) ) )
#criando novo df com os filtros do período de vigência da sanção aplicados
df_sancoes = df_sancoes_cnpj[condicao1 | condicao2]

#**************************************************
# ATENÇÃO!!!!! FUTURO: impedimento só não pode contratar com empresa no CEIS se o órgão comprador for da MESMA ESFERA do órgão que sancionou. 
# Então teria que avaliar a categoria da sanção e comparar com a esfera do órgão para saber!!!!!
#**************************************************

In [18]:
#df_sancoes.info()

#verificação: tem numcontrolePNCP com mais de 1 ocorrência (2) porque a empresa contratada tem mais de 1 sanção (2) vigentes para o período de assinatura do contrato
#(são poucas ocorrências: apenas em 4 contratos)
#df_sancoes.groupby(['numeroControlePNCP'])['numeroControlePncpCompra'].count().sort_values(ascending=False)

# filtro = df_sancoes['numeroControlePNCP'] =='00394452054700-2-003014/2023'
# df_filtrado = df_sancoes[filtro]
# df_filtrado
#df_temp[['numeroControlePNCP', 'orgaoEntidade_razaoSocial', 'unidadeOrgao_nomeUnidade', 'dataAssinatura', 'DATA INÍCIO SANÇÃO', 'DATA FINAL SANÇÃO', 'niFornecedor', 'nomeRazaoSocialFornecedor']]

#1879 ocorrências
df_sancoes[['numeroControlePNCP', 'orgaoEntidade_razaoSocial', 
            'unidadeOrgao_nomeUnidade', 'unidadeOrgao_ufSigla', 'unidadeOrgao_municipioNome', 
            'niFornecedor', 'nomeRazaoSocialFornecedor',
            'dataAssinatura', 'DATA INÍCIO SANÇÃO', 'DATA FINAL SANÇÃO']].head()

,numeroControlePNCP,orgaoEntidade_razaoSocial,unidadeOrgao_nomeUnidade,unidadeOrgao_ufSigla,unidadeOrgao_municipioNome,niFornecedor,nomeRazaoSocialFornecedor,dataAssinatura,DATA INÍCIO SANÇÃO,DATA FINAL SANÇÃO
95,00394502000144-2-004037/2023,COMANDO DA MARINHA,CAPITANIA DOS PORTOS DE MACAÉ,RJ,Macaé,30680100000177,JTH COMERCIO LTDA,2023-07-12,2023-04-25,25/04/2025
96,88830609000139-2-000158/2023,MUNICIPIO DE CAXIAS DO SUL,Executivo Administração Direta,RS,Caxias do Sul,30680100000177,JTH COMERCIO LTDA,2023-08-21,2023-04-25,25/04/2025
97,17217985000104-2-000301/2023,UNIVERSIDADE FEDERAL DE MINAS GERAIS,FACULDADE DE MEDICINA/UFMG,MG,Belo Horizonte,30680100000177,JTH COMERCIO LTDA,2023-07-17,2023-04-25,25/04/2025
98,33287806000161-2-000005/2023,CONSELHO REGIONAL DE CONTABILIDADE DO ESTADO D...,CONSELHO REGIONAL DE CONTABILIDADE - RJ,RJ,Rio de Janeiro,30680100000177,JTH COMERCIO LTDA,2023-08-16,2023-04-25,25/04/2025
99,00509968000148-2-001106/2023,TRIBUNAL SUPERIOR DO TRABALHO,TRIBUNAL REGIONAL DO TRABALHO DA 17A. REGIAO,ES,Vitória,30680100000177,JTH COMERCIO LTDA,2023-08-21,2023-04-25,25/04/2025


In [19]:
# 1. Verificando quais são as EMPRESAS distintas com sanção: tem 87 empresas com sanção vigente e com contratos com a adm
# Será usado no GRAFICO A
df_empresas_qtdcontratos = df_sancoes.groupby(['niFornecedor','nomeRazaoSocialFornecedor'])['numeroControlePNCP'].count().sort_values(ascending=False) 
df_empresas_qtdcontratos = df_empresas_qtdcontratos.reset_index()

empresasSancao_media = math.ceil(df_empresas_qtdcontratos.numeroControlePNCP.mean())  #média: 21.6 - arredondado: 22
empresasSancao_mediana = math.ceil(df_empresas_qtdcontratos.numeroControlePNCP.median())  #mediana: 2.0 - arredondado: 2
empresasSancao_totalempresas = df_empresas_qtdcontratos.numeroControlePNCP.count()

df_empresas_qtdcontratos.head(10) #10 maiores 

#testando se o retorno está correto
#exemplo da empresa que tem sanção vigente e que tem mais contratos assinados (mais de 2000 contratos com sanção!!!)
# filtro1 = df_sancoes['niFornecedor'] == '12889035000102'
# df_filtrado1 = df_sancoes[filtro1]
# df_filtrado1
#filtro2 = df_contrato['niFornecedor'] == '12889035000102'
#df_filtrado2 = df_contrato[filtro2]
#df_filtrado2

#df_sancoes.groupby(['niFornecedor','nomeRazaoSocialFornecedor'])['numeroControlePNCP'].value_counts().sort_values(ascending=False)
#exemplo de 1 registro que para o mesmo contrato (numPNCP) tem 2 linhas, porque tem 2 sanções vigente para a empresa no período
# filtro = df_sancoes['numeroControlePNCP'] == '27142058001521-2-000008/2023'
# df_filtrado = df_sancoes[filtro]
# df_filtrado

#*** obs.: ***
#count() fornece a contagem total de elementos, incluindo valores nulos, enquanto 
#value_counts() fornece a contagem de valores únicos, excluindo valores nulos.

,niFornecedor,nomeRazaoSocialFornecedor,numeroControlePNCP
0,40738368000176,GAMBA CONEXOES COMERCIO ATACADISTA DE ALIMENTO...,961
1,12889035000102,INOVAMED HOSPITALAR LTDA,421
2,44922093000187,LUMA DE OLIVEIRA RAMBO LTDA,144
3,25279552000101,DISTRIBUIDORA DE MEDICAMENTOS BACKES LTDA,40
4,10869890000126,AR FIORENZANO DISTRIBUIDORA DE MEDICAMENTOS LTDA,35
5,25279552000101,DISTRIBUIDORA DE MEDICAMENTOS BACKES EIRELI,32
6,28480081000193,INOVA LASER E COMUNICACAO VISUAL LTDA,20
7,29700587000123,PRIORITTA PRODUTOS HOSPITALARES LTDA,18
8,21767486000168,R M LANZA DOS SANTOS COMERCIO EPP,15
9,26889274000177,S.M GUIMARAES DISTRIBUIDORA IMPORTADORA E EXPO...,14


In [20]:
# 2. Verificando quais são os ÓRGÃOS que tem contratos com empresas sancionadas: 365 órgãos
# Será usado no GRÁFICO B
# obs.: o nome do órgão agrupa várias unidades diferentes, por isso tem o CNPJ junto

df_orgaos_qtdcontratos = df_sancoes.groupby(['orgaoEntidade_cnpj', 'orgaoEntidade_razaoSocial', 'orgaoEntidade_esferaId'])['numeroControlePNCP'].count().sort_values(ascending=False) 
df_orgaos_qtdcontratos = df_orgaos_qtdcontratos.reset_index()

orgaosSancao_media = math.ceil(df_orgaos_qtdcontratos.numeroControlePNCP.mean())  #média: 6
orgaosSancao_mediana = math.ceil(df_orgaos_qtdcontratos.numeroControlePNCP.median())  #mediana: 3
orgaosSancao_totalorgaos = df_orgaos_qtdcontratos.numeroControlePNCP.count() #365 orgaos

#converte cnpj para string para poder concatenar
df_orgaos_qtdcontratos['orgaoEntidade_cnpj'] = df_orgaos_qtdcontratos['orgaoEntidade_cnpj'].astype(str) #converte CNPJ para string
#combinando orgaoEntidade_razaoSocial e orgaoEntidade_cnpj em uma nova coluna
df_orgaos_qtdcontratos['orgaoEntidade'] = df_orgaos_qtdcontratos['orgaoEntidade_razaoSocial'] + ' (' + df_orgaos_qtdcontratos['orgaoEntidade_cnpj'] + ')'

#df_orgaos_qtdcontratos.info()
df_orgaos_qtdcontratos.head(10)  #10 orgaos com mais contratos

# filtro = df_sancoes['orgaoEntidade_razaoSocial'] == 'FUNDO MUNICIPAL DE SAUDE'
# df_filtrado = df_sancoes[filtro]
# df_filtrado[['orgaoEntidade_cnpj', 'orgaoEntidade_razaoSocial', 'unidadeOrgao_nomeUnidade']][0:]

,orgaoEntidade_cnpj,orgaoEntidade_razaoSocial,orgaoEntidade_esferaId,numeroControlePNCP,orgaoEntidade
0,1611828000149,MUNICIPIO DE NOVA RAMADA,M,138,MUNICIPIO DE NOVA RAMADA (1611828000149)
1,15126437000143,EMPRESA BRASILEIRA DE SERVIÇOS HOSPITALARES,F,35,EMPRESA BRASILEIRA DE SERVIÇOS HOSPITALARES (1...
2,75438655000145,MUNICIPIO DE CORREIA PINTO,M,24,MUNICIPIO DE CORREIA PINTO (75438655000145)
3,78510112000180,MUNICIPIO DE LINDOIA DO SUL,M,19,MUNICIPIO DE LINDOIA DO SUL (78510112000180)
4,83754044000134,MUNICIPIO DE CURITIBANOS,M,19,MUNICIPIO DE CURITIBANOS (83754044000134)
5,82892308000153,MUNICIPIO DE BIGUACU,M,18,MUNICIPIO DE BIGUACU (82892308000153)
6,1613428000172,MUNICIPIO DE LUZERNA,M,18,MUNICIPIO DE LUZERNA (1613428000172)
7,83074302000131,MUNICIPIO DE CACADOR,M,18,MUNICIPIO DE CACADOR (83074302000131)
8,83039842000184,MUNICIPIO DE VIDEIRA,M,17,MUNICIPIO DE VIDEIRA (83039842000184)
9,394452000103,COMANDO DO EXERCITO,F,17,COMANDO DO EXERCITO (394452000103)


In [21]:
#3. verificando POR EMPRESA, em QUAIS UFs e quantos municípios atua, e com que qtd de contratos
#Será usado no GRÁFICO E
agrupamento = df_sancoes.groupby('nomeRazaoSocialFornecedor').agg({
    'unidadeOrgao_ufSigla': lambda x: x.unique().tolist(),
    'unidadeOrgao_municipioNome': 'nunique',  # Conta municípios distintos
    'numeroControlePNCP': 'count'  # Conta contratos
}).reset_index()

# Renomeia as colunas
agrupamento.rename(columns={
    'nomeRazaoSocialFornecedor': 'Fornecedor',
    'unidadeOrgao_ufSigla': 'UF',
    'unidadeOrgao_municipioNome': 'Qtd Municípios',
    'numeroControlePNCP': 'Qtd Contratos'
}, inplace=True)

agrupamento = agrupamento.sort_values(by='Qtd Contratos', ascending=False)
agrupamento_top10 = agrupamento[0:10]

agrupamento_top10
#agrupamento

,Fornecedor,UF,Qtd Municípios,Qtd Contratos
32,GAMBA CONEXOES COMERCIO ATACADISTA DE ALIMENTO...,[SC],181,961
43,INOVAMED HOSPITALAR LTDA,"[RS, SC, PR, MG, PI]",183,421
49,LUMA DE OLIVEIRA RAMBO LTDA,[RS],2,144
21,DISTRIBUIDORA DE MEDICAMENTOS BACKES LTDA,"[SC, RS]",29,40
8,AR FIORENZANO DISTRIBUIDORA DE MEDICAMENTOS LTDA,[SC],30,35
20,DISTRIBUIDORA DE MEDICAMENTOS BACKES EIRELI,[SC],25,32
38,INOVA LASER E COMUNICACAO VISUAL LTDA,"[PR, MG, RS, MA, TO, PB, SP, RJ, MS]",11,20
66,PRIORITTA PRODUTOS HOSPITALARES LTDA,"[SC, RS]",17,18
69,R M LANZA DOS SANTOS COMERCIO EPP,[MG],1,15
70,S.M GUIMARAES DISTRIBUIDORA IMPORTADORA E EXPO...,[RS],1,14


In [22]:
#4. para GRÁFICO D
#verificando em que MUNICÍPIOS estão as unidades com mais contratos com empresas com sanções: 261 municípios

#filtrando apenas sancoes em órgãos municipais (e não estaduais ou federais)
filtro = df_sancoes['orgaoEntidade_esferaId'] == 'M' 
df_sancoes_somunicipios = df_sancoes[filtro]

df_municipios_qtdcontratos = df_sancoes_somunicipios.groupby(['unidadeOrgao_ufSigla', 'unidadeOrgao_codigoIbge', 'unidadeOrgao_municipioNome'])['numeroControlePNCP'].count().sort_values(ascending=False)  
df_municipios_qtdcontratos = df_municipios_qtdcontratos.reset_index()

#converte codIBGE para string para usar função que usa esse codigo no mapa de calor
#df_municipios_qtdcontratos['unidadeOrgao_codigoIbge'] = df_municipios_qtdcontratos['unidadeOrgao_codigoIbge'].astype(str) #converte codIBGE para string

#df_municipios_qtdcontratos.info()
df_municipios_qtdcontratos.head(10)


,unidadeOrgao_ufSigla,unidadeOrgao_codigoIbge,unidadeOrgao_municipioNome,numeroControlePNCP
0,RS,4313334,Nova Ramada,138
1,SC,4209300,Lages,28
2,SC,4204558,Correia Pinto,24
3,SC,4202305,Biguaçu,23
4,SC,4213906,Presidente Castello Branco,22
5,SC,4216909,São Lourenço do Oeste,21
6,SC,4211306,Navegantes,19
7,SC,4203006,Caçador,19
8,SC,4208906,Jaraguá do Sul,19
9,SC,4206702,Herval D'Oeste,19


In [23]:
## verificando que df_contrato só possui 1 linha para cada numeroControlePNCP
#df_contrato.groupby(['numeroControlePNCP'])['numeroControlePncpCompra'].count().sort_values(ascending=False)

#verificando que df_sancoes pode ter mais de 1 linha para cada numeroControlePNCP, pois um CNPJ pode ter várias sanções diferentes
#df_sancoes.groupby(['numeroControlePNCP'])['numeroControlePncpCompra'].count().sort_values(ascending=False)

#df_sancoes_filtrado_data.groupby(['numeroControlePNCP'])['numeroControlePncpCompra'].count().sort_values(ascending=False)

#df.unique()

In [24]:
#GRAFICO A só com TOP 10 EMPRESAS
#1. GRAFICO A (usando PLOTLI) - gráfico de barras (vertical) 
# Visão das 10 EMPRESAS (mais) sancionadas, com qtd de contratos de cada uma
df_empresas_qtdcontratos_top10 = df_empresas_qtdcontratos.head(10).copy()
df_empresas_qtdcontratos_top10 = df_empresas_qtdcontratos_top10.reset_index()

# Selecione as empresas e contratos (substitua pelos seus próprios dados)
empresas = df_empresas_qtdcontratos_top10['nomeRazaoSocialFornecedor']
contratos = df_empresas_qtdcontratos_top10['numeroControlePNCP']

fig = go.Figure()

# Adicione as barras ao gráfico
fig.add_trace(go.Bar(x=empresas, y=contratos, marker_color='skyblue'))

# Adicione a anotação de texto (números referentes à quantidade de contratos)
for empresa, contrato in zip(empresas, contratos):
    fig.add_annotation(
        text=str(contrato),
        x=empresa,
        y=contrato,
        showarrow=False,
        font=dict(size=12)
    )

fig.update_xaxes(categoryorder='total descending')

#adiciona a MÉDIA como uma linha horizontal
fig.add_shape(
    type='line',
    x0=0,  # Início da linha no eixo X (posição inicial)
    x1=len(empresas),  # Fim da linha no eixo X (posição final)
    y0=empresasSancao_media,  # Valor da média
    y1=empresasSancao_media,  # Valor da média
    line=dict(
        color='red',  # Cor da linha
        width=2,  # Largura da linha
        dash='dash'  # Estilo da linha (pode ser 'solid', 'dash', 'dot', etc.)
    )
)

# Adicione uma anotação para mostrar o número da média ao final da linha
fig.add_annotation(
    text=f'Média: {empresasSancao_media:}',  # Formatação para duas casas decimais
    xref='paper',  # Define a referência para coordenadas em relação ao papel
    yref='y',  # Define a referência para coordenadas no eixo Y
    x=1.1,  # Posição X da anotação (ajuste conforme necessário)
    y=empresasSancao_media,  # Posição Y da anotação
    showarrow=False,  # Não mostrar seta de destaque
    font=dict(size=12, color='red')  # Tamanho da fonte do texto
)

fig.update_layout(
    title=' Top 10 fornecedores sancionados com mais contratos <br>(total de fornecedores sancionados: <b>' + str(empresasSancao_totalempresas) + '</b>)',
    xaxis_title='Fornecedor com sanção(s)',
    yaxis_title='Número de Contratos',
    xaxis_tickangle=-45,
    title_x=0.8, # Define o alinhamento do título no centro
    yaxis=dict(tick0=0, dtick=500),  # Valor inicial do eixo Y = 0, Intervalo entre os valores do eixo Y = 500
    height=768,
    width=1000
)

fig.show()

In [25]:
#1.1 GRAFICO A só com TOP 10 EMPRESAS - SUSANDO LOLIPOP CHART
# Visão das 10 EMPRESAS (mais) sancionadas, com qtd de contratos de cada uma

# Selecione as empresas e contratos (substitua pelos seus próprios dados)
empresas = df_empresas_qtdcontratos_top10['nomeRazaoSocialFornecedor']
contratos = df_empresas_qtdcontratos_top10['numeroControlePNCP']

fig = go.Figure()

# Adicione os marcadores estilo lollipop
fig.add_trace(go.Scatter(
    x=empresas,  # Posições X das empresas
    y=contratos,  # Número de contratos
    mode='markers+lines',  # Modo de marcadores com linhas conectando-os
    marker=dict(size=10, color='skyblue'),  # Tamanho e cor dos marcadores
    line=dict(width=2, color='skyblue'),  # Estilo da linha conectando os marcadores
    text=contratos,  # Texto que aparecerá ao passar o mouse sobre os marcadores
    textposition='top center',  # Posição do texto
))

# Adicione a média como uma linha horizontal
fig.add_shape(
    type='line',
    x0=0,  # Início da linha no eixo X (posição inicial)
    x1=len(empresas),  # Fim da linha no eixo X (posição final)
    y0=empresasSancao_media,  # Valor da média
    y1=empresasSancao_media,  # Valor da média
    line=dict(
        color='red',  # Cor da linha
        width=2,  # Largura da linha
        dash='dash'  # Estilo da linha (pode ser 'solid', 'dash', 'dot', etc.)
    )
)

# Adicione uma anotação para mostrar o número da média ao final da linha
fig.add_annotation(
    text=f'Média: {empresasSancao_media:.2f}',  # Formatação para duas casas decimais
    xref='paper',  # Define a referência para coordenadas em relação ao papel
    yref='y',  # Define a referência para coordenadas no eixo Y
    x=1.1,  # Posição X da anotação (ajuste conforme necessário)
    y=empresasSancao_media,  # Posição Y da anotação
    showarrow=False,  # Não mostrar seta de destaque
    font=dict(size=12, color='red')  # Tamanho e cor da fonte do texto
)

fig.update_layout(
    title=' Top 10 fornecedores sancionados com mais contratos <br>(total de fornecedores sancionados: <b>' + str(empresasSancao_totalempresas) + '</b>)',
    xaxis_title='Fornecedor com sanção(s)',
    yaxis_title='Número de Contratos',
    xaxis_tickangle=-45,
    title_x=0.8, # Define o alinhamento do título no centro
    yaxis=dict(tick0=0, dtick=500),  # Valor inicial do eixo Y = 0, Intervalo entre os valores do eixo Y = 500
    height=768,
    width=1000
)

fig.show()

In [26]:
#1.1 GRAFICO A - EMPRESAS - Alternativa usando grafico TREEMAP

# Organize os dados hierarquicamente, como um DataFrame de treemap requer
df_treemap = df_empresas_qtdcontratos.sort_values('numeroControlePNCP', ascending=False)
df_treemap['Hierarquia'] = 'Empresas com Sanções'
df_treemap = df_treemap.rename(columns={'nomeRazaoSocialFornecedor': 'Nome da Empresa', 'numeroControlePNCP': 'Número de Contratos'})

fig = px.treemap(df_treemap, 
                 path=['Hierarquia', 'Nome da Empresa'], 
                 values='Número de Contratos',
                 title='Número de Contratos por Empresa com Sanção')
fig.update_traces(textinfo='label+value')
fig.show()

In [27]:
#1.2 GRÁFICO E - TREEMAP COM CATEGORIAS (SUBNÍVEIS) DE UF DENTRO DA EMPRESA
# Visão das 10 empresas com mais contratos, demonstranto as UFs onde tem mais contratos

# Agrupe os dados e renomeie as colunas
agrupamento = df_sancoes.groupby(['nomeRazaoSocialFornecedor', 'unidadeOrgao_ufSigla']).agg({
    'unidadeOrgao_municipioNome': 'nunique',
    'numeroControlePNCP': 'count'
}).reset_index()

agrupamento.rename(columns={
    'nomeRazaoSocialFornecedor': 'Fornecedor',
    'unidadeOrgao_ufSigla': 'UF',
    'unidadeOrgao_municipioNome': 'Qtd Municípios',
    'numeroControlePNCP': 'Qtd Contratos'
}, inplace=True)

agrupamento = agrupamento.sort_values(by='Qtd Contratos', ascending=False)

# Crie um treemap com a hierarquia 'Fornecedor' > 'UF'
fig = px.treemap(
    agrupamento,
    path=['Fornecedor', 'UF'],  # Hierarquia de categorias
    values='Qtd Contratos',  # Valores para determinar o tamanho das áreas
    color='Qtd Contratos',  # Cor das áreas baseada nos contratos
    color_continuous_scale='Viridis',  # Escolha o mapa de cores desejado
    title='Top Fornecedores com suas UFs e Contratos',
    width=1000,
    height=600
)

fig.show()

In [28]:
#1.3 GRÁFICO E - TREEMAP COM CATEGORIAS (SUBNÍVEIS) DE EMPRESA DENTRO DA uf
# Visão das UFs onde tem mais contratos com as empresas dentro

# Agrupe os dados e renomeie as colunas
agrupamento = df_sancoes.groupby(['nomeRazaoSocialFornecedor', 'unidadeOrgao_ufSigla']).agg({
    'unidadeOrgao_municipioNome': 'nunique',
    'numeroControlePNCP': 'count'
}).reset_index()

agrupamento.rename(columns={
    'nomeRazaoSocialFornecedor': 'Fornecedor',
    'unidadeOrgao_ufSigla': 'UF',
    'unidadeOrgao_municipioNome': 'Qtd Municípios',
    'numeroControlePNCP': 'Qtd Contratos'
}, inplace=True)

agrupamento = agrupamento.sort_values(by='Qtd Contratos', ascending=False)

# Crie um treemap com a hierarquia 'Fornecedor' > 'UF'
fig = px.treemap(
    agrupamento,
    path=['UF', 'Fornecedor'],  # Hierarquia de categorias
    values='Qtd Contratos',  # Valores para determinar o tamanho das áreas
    color='Qtd Contratos',  # Cor das áreas baseada nos contratos
    color_continuous_scale='Viridis',  # Escolha o mapa de cores desejado
    title='Top UFs com Contratos com empresas sancionadas',
    width=1000,
    height=600
)

fig.show()

In [29]:
# 2. GRAFICO B (PLOTLI) - gráfico de barras horizontais mostrando apenas TOP 10 ÓRGÃOS COM MAIS CONTRATOS
# Visão dos TOP 10 ÓRGÃOS com mais contratos com empresas sancionadas

df_orgaos_qtdcontratos_top10 = df_orgaos_qtdcontratos.head(10).copy() #JBX!!!!
df_orgaos_qtdcontratos_top10 = df_orgaos_qtdcontratos_top10.reset_index()

# para exibir o eixo y de 0 até quem tem mais contratos
df_orgaos_qtdcontratos_top10 = df_orgaos_qtdcontratos_top10.sort_values('numeroControlePNCP')

orgaos = df_orgaos_qtdcontratos_top10['orgaoEntidade']
contratos = df_orgaos_qtdcontratos_top10['numeroControlePNCP']

fig = px.bar(
    df_orgaos_qtdcontratos_top10, 
    x=contratos, 
    y=orgaos, 
    orientation='h',
    text=contratos,  # Agora o texto mostrará o número de contratos
)

# Adicione uma linha vertical para a média
fig.add_shape(
    type='line',
    x0=orgaosSancao_mediana,  # Posição vertical da média
    x1=orgaosSancao_mediana,  # Posição vertical da média
    y0=0,  # Início da linha no eixo Y (posição inicial)
    y1=len(orgaos),  # Fim da linha no eixo Y (posição final)
    line=dict(
        color='red',
        width=2,
        dash='dash'
    )
)

# Adicione uma anotação para mostrar o número da média ao final da linha da média
fig.add_annotation(
    text=f'mediana: {orgaosSancao_mediana:}',  # Formatação para duas casas decimais
    xref='x',  # Define a referência para coordenadas no eixo X
    yref='y',  # Define a referência para coordenadas no eixo Y
    x=orgaosSancao_mediana,  # Posição X da anotação
    y=len(orgaos),  # Posição Y da anotação
    showarrow=True,  # Mostrar uma seta de destaque
    arrowhead=2,  # Estilo da seta (pode ser ajustado conforme necessário)
    font=dict(size=12, color='red')  # Tamanho da fonte do texto
)

fig.update_layout(
    title='Top 10 Órgãos/Entidades com mais contratos com Empresas Sancionadas',
    xaxis_title='Número de Contratos',
    yaxis_title='Órgão/Entidade',
    title_x=0.5,
    xaxis=dict(tick0=0, dtick=500)
)

fig.show()

In [32]:
#2.1 GRAFICO BOXPLOT
#Visão dos órgãos com qtd de contratos com empresas sancionadas por esfera

# ATENÇÃO: ******* excluindo o registro da esfera esquisita = 'N':
df_orgaos_qtdcontratos_excetoEsferaN = df_orgaos_qtdcontratos[df_orgaos_qtdcontratos['orgaoEntidade_esferaId'] != 'N']

# ATENÇÃO: ******* excluindo o órgão com o maior número de contratos (porque é outlier - extraoficialmente: na função do gráfico que identifica outlier, não foi considerado outlier
orgao_maior_contrato = df_orgaos_qtdcontratos_excetoEsferaN['numeroControlePNCP'].idxmax()
# Exclua esse órgão do DataFrame
df_orgaos_qtdcontratos_excetoEsferaN.drop(orgao_maior_contrato, inplace=True)

#excluindo os outliers
#fig.update_traces(boxpoints='outliers')  # Mostra apenas os outliers nos boxplots
#fig.update_traces(boxpoints=False)  # Remove os outliers dos boxplots

fig = px.box(df_orgaos_qtdcontratos_excetoEsferaN, x='orgaoEntidade_esferaId', y='numeroControlePNCP', color='orgaoEntidade_esferaId',
             labels={'orgaoEntidade_esferaId': 'Esfera', 'numeroControlePNCP': 'Número de Contratos'},
             title='Distribuição da Quantidade de Contratos por Esfera e Órgão')

fig.update_traces(boxpoints='all')  # Mostra todos os pontos nos boxplots
fig.update_layout(height=600,  # Defina a altura desejada em pixels
    title='Distribuição da Quantidade de Contratos por Esfera e Órgão'
)
fig.show()

In [ ]:
#3. GRÁFICO C - Gráfico de dispersão
# Cada ponto no gráfico representa uma empresa sancionada, e sua posição é determinada pelo número de contratos.

fig = px.scatter(df_empresas_qtdcontratos, 
                 x='numeroControlePNCP', 
                 y='nomeRazaoSocialFornecedor', 
                 title='Gráfico de Dispersão: Número de Contratos por Empresa Sancionada')

fig.update_traces(marker=dict(size=12, opacity=0.6))
fig.update_layout(xaxis_title='Número de Contratos', 
                  yaxis_title='Nome da Empresa',
                  height=1500,
                  width=1500)
fig.show()

In [36]:
#E. GRAFICO - MAPA (COM BOLINHAS)
# Visão dos municípios (das unidades executaoras) com mais contratos com empresas sancionadas

#buscando latitude e longitude dos municipios no csv correspondente
df_latlong = pd.read_csv('dados/IBGE_MunicipiosLatLong.csv',sep=";", encoding="latin1")

# Merge dos DataFrames com base na coluna 'codIBGE'
df_merged = df_municipios_qtdcontratos.merge(df_latlong, left_on='unidadeOrgao_codigoIbge', right_on='GEOCODIGO_MUNICIPIO')
df_merged['marker_size'] = df_merged['numeroControlePNCP'] * 1000

# Ajuste o tamanho e a cor dos marcadores, bem como a escala de cores
fig = px.scatter_geo(df_merged,
                     lat='LATITUDE',
                     lon='LONGITUDE',
                     text='numeroControlePNCP',
                     size='marker_size',  # Ajuste o tamanho com base nos contratos
                     projection='natural earth',
                     title='Mapa de Calor: Municípios com contratos com empresas sancionadas')

fig.update_geos(showcoastlines=True, coastlinecolor="Black", showland=True, landcolor="white")
fig.update_layout(geo=dict(scope='south america'))
fig.update_layout(width=1000, height=700)

# Ajuste a escala de cores e o tamanho da escala dos marcadores
fig.update_traces(marker=dict(showscale=True, colorscale='Viridis', size=10))  # Ajuste o tamanho conforme necessário

# Adicione rótulos de texto para destacar os pontos
fig.update_traces(textposition='top center', textfont_size=10)

fig.show(config={"displayModeBar": False})